<a href="https://colab.research.google.com/github/hck717/Rotation-ETF/blob/main/Rotation_ETF_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas fredapi tweepy vaderSentiment alpha_vantage pytrends

In [ ]:
! pip install ta-lib-bin > log.txt

In [ ]:
pip install yfinance==0.2.54 pandas_datareader fredapi pytrends pandas numpy requests beautifulsoup4

Data Collection

In [ ]:
FRED_API_KEY = os.getenv("FRED_API_KEY", "2e978cd1b8ac041452388ed44ac1dcd6")

In [ ]:
import yfinance as yf
import pandas as pd
import pandas_datareader as pdr
from fredapi import Fred
import numpy as np
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup
import time
import os

# Define ETF universe
ETF_UNIVERSE = [
    "MTUM", "VLUE", "QUAL", "USMV",  # U.S. Factor ETFs
    "XLK", "XLV", "XLE", "XLU",      # U.S. Sectors
    "EEM", "EFA",                    # Global
    "TLT", "IEF", "TIP",             # Fixed Income
    "BIL"                            # Cash Proxy
]

# Cross-asset tickers
CROSS_ASSETS = ["SPY", "TLT", "GLD"]

# Date range for data collection
START_DATE = "2010-01-01"
END_DATE = datetime.today().strftime("%Y-%m-%d")

# Directory to store CSV files
DATA_DIR = "etf_data"
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

# Initialize FRED API with your provided API key
fred = Fred(api_key="2e978cd1b8ac041452388ed44ac1dcd6")

# Toggle for Google Trends usage (set to False to disable for now)
use_google_trends = False  # Change to True to enable Google Trends data collection

# Helper function to save DataFrame to CSV
def save_to_csv(df, file_name):
    file_path = os.path.join(DATA_DIR, f"{file_name}.csv")
    df.to_csv(file_path)
    print(f"Saved {file_name} to {file_path}")

# 1. Price and Volume Data (yfinance)
def collect_price_volume_data(tickers, retries=3):
    price_data = {}
    volume_data = {}
    avg_daily_volume = {}
    failed_tickers = []
    missing_data_tickers = []

    for ticker in tickers:
        print(f"Collecting price/volume data for {ticker}")
        attempt = 0
        success = False
        while attempt < retries and not success:
            try:
                data = yf.download(ticker, start=START_DATE, end=END_DATE, auto_adjust=False, progress=False)
                if data.empty or 'Adj Close' not in data.columns:
                    raise ValueError(f"No data or 'Adj Close' missing for {ticker}")

                # Check for missing data in early dates
                if data.iloc[0:].isna().all().all():
                    missing_data_tickers.append(ticker)
                else:
                    price_data[ticker] = data[["Open", "High", "Low", "Close", "Adj Close"]]
                    volume_data[ticker] = data["Volume"]
                    recent_volume = data["Volume"][-30:]
                    avg_daily_volume[ticker] = recent_volume.mean() if not recent_volume.empty else np.nan
                    success = True
            except Exception as e:
                print(f"Error collecting data for {ticker} (attempt {attempt + 1}/{retries}): {e}")
                attempt += 1
                time.sleep(2 ** attempt)
        if not success:
            print(f"Failed to collect data for {ticker} after {retries} attempts.  Skipping.")
            failed_tickers.append(ticker)

    if not price_data:
        print("Warning: No price data collected for any ticker.")
        return pd.DataFrame(), pd.DataFrame(), pd.Series(), failed_tickers, missing_data_tickers

    price_df = pd.concat(price_data, axis=1)
    volume_df = pd.concat(volume_data, axis=1)
    avg_volume_df = pd.Series(avg_daily_volume, name="Avg_Daily_Volume")

    save_to_csv(price_df, "price_data")
    save_to_csv(volume_df, "volume_data")
    save_to_csv(avg_volume_df, "avg_daily_volume")

    return price_df, volume_df, avg_volume_df, failed_tickers, missing_data_tickers

# 2. Factor-Specific Data
def collect_factor_specific_data(price_df, successful_tickers):
    factor_data = {}

    if price_df.empty:
        print("No price data available for factor-specific calculations.")
        return pd.DataFrame()

    for ticker in successful_tickers:
        print(f"Calculating factor-specific data for {ticker}")
        try:
            ticker_data = price_df[ticker]["Adj Close"]
            ticker_data = ticker_data.dropna()

            if len(ticker_data) < 252:  # Need at least 252 days for 12-month momentum
                print(f"Skipping {ticker}: Insufficient data for momentum calculation.")
                continue

            # Momentum: 12-month return
            momentum = ticker_data.pct_change(periods=252).dropna()
            factor_data[f"{ticker}_momentum"] = momentum

            # Volatility: 30-day rolling standard deviation
            volatility = ticker_data.pct_change().rolling(window=30).std() * np.sqrt(252)
            factor_data[f"{ticker}_volatility"] = volatility
        except Exception as e:
            print(f"Error calculating factor-specific data for {ticker}: {e}")

    if not factor_data:
        print("Warning: No factor-specific data collected.")
        return pd.DataFrame()

    factor_df = pd.concat(factor_data, axis=1)
    save_to_csv(factor_df, "factor_data")

    return factor_df

# 3. Macroeconomic Data (FRED API) - Fixed PMI issue
def collect_macro_data():
    macro_series = {
        "CPI_YoY": "CPIAUCSL",
        "10Y_2Y_Spread": "T10Y2Y",
        "Unemployment_Rate": "UNRATE",
        "TED_Spread": "TEDRATE",
        "Industrial_Production": "INDPRO"
    }

    macro_data = {}
    unavailable_series = []
    for name, series_id in macro_series.items():
        print(f"Collecting macro data: {name}")
        try:
            data = fred.get_series(series_id, START_DATE, END_DATE)
            data.index = pd.to_datetime(data.index)
            macro_data[name] = data
        except Exception as e:
            print(f"Error collecting {name}: {e}")
            unavailable_series.append(name)

    # Calculate YoY CPI change
    if "CPI_YoY" in macro_data:
        macro_data["CPI_YoY"] = macro_data["CPI_YoY"].pct_change(periods=12) * 100

    if not macro_data:
        print("Warning: No macro data collected.")
        return pd.DataFrame()

    if unavailable_series:
        print(f"Unavailable macro series: {unavailable_series}")

    macro_df = pd.concat(macro_data, axis=1)
    macro_df = macro_df.ffill()  # Forward-fill to align frequencies
    save_to_csv(macro_df, "macro_data")

    return macro_df

# 4. Volatility Data
def collect_volatility_data():
    print("Collecting VIX data")
    try:
        vix_data = yf.download("^VIX", start=START_DATE, end=END_DATE, auto_adjust=False, progress=False)
        if vix_data.empty or 'Adj Close' not in vix_data.columns:
            raise ValueError("No VIX data or 'Adj Close' missing")
        vix_df = pd.DataFrame(vix_data["Adj Close"], columns=["VIX"])
        save_to_csv(vix_df, "volatility_data")
        return vix_df
    except Exception as e:
        print(f"Error collecting VIX data: {e}")
        return pd.DataFrame()

# 5. Alternative Data
def collect_alternative_data(price_df):
    # Google Trends data collection (disabled by default)
    google_trends_df = pd.DataFrame()
    if use_google_trends:
        from pytrends.request import TrendReq
        pytrends = TrendReq(hl="en-US", tz=360)
        keywords = {
            "XLK": "tech stocks",
            "XLE": "energy crisis",
            "XLV": "healthcare stocks",
            "XLU": "utilities stocks"
        }
        google_trends_data = {}
        for ticker, keyword in keywords.items():
            print(f"Collecting Google Trends data for {keyword}")
            retries = 3
            for attempt in range(retries):
                try:
                    pytrends.build_payload([keyword], timeframe=f"{START_DATE} {END_DATE}")
                    data = pytrends.interest_over_time()
                    if not data.empty:
                        google_trends_data[ticker] = data[keyword]
                    break
                except Exception as e:
                    print(f"Error collecting Google Trends for {keyword} (attempt {attempt + 1}/{retries}): {e}")
                    if "code 429" in str(e):
                        time.sleep(60)
                    else:
                        break

        if google_trends_data:
            google_trends_df = pd.concat(google_trends_data, axis=1)
            save_to_csv(google_trends_df, "google_trends_data")
        else:
            print("Warning: No Google Trends data collected.")

    # Credit Spreads (FRED)
    print("Collecting credit spreads")
    try:
        credit_spread = fred.get_series("BAMLH0A0HYM2", START_DATE, END_DATE)
        credit_spread_df = pd.DataFrame(credit_spread, columns=["Credit_Spread"])
        credit_spread_df.index = pd.to_datetime(credit_spread_df.index)
        if not price_df.empty:
            credit_spread_df = credit_spread_df.reindex(price_df.index, method='ffill')
        save_to_csv(credit_spread_df, "credit_spreads")
    except Exception as e:
        print(f"Error collecting credit spreads: {e}")
        credit_spread_df = pd.DataFrame()

    # Sentiment Scores (Placeholder aligned with price data index)
    sentiment_data = {}
    if not price_df.empty:
        date_range = price_df.index
        for ticker in ETF_UNIVERSE:
            sentiment_data[ticker] = pd.Series(np.random.uniform(0, 1, len(date_range)), index=date_range)
        sentiment_df = pd.DataFrame(sentiment_data)
        save_to_csv(sentiment_df, "sentiment_data")
    else:
        sentiment_df = pd.DataFrame()
        print("Warning: No sentiment data generated due to empty price data.")

    return google_trends_df, credit_spread_df, sentiment_df

# 6. Cross-Asset Data
def collect_cross_asset_data():
    price_data = {}
    for ticker in CROSS_ASSETS:
        print(f"Collecting cross-asset data for {ticker}")
        try:
            data = yf.download(ticker, start=START_DATE, end=END_DATE, auto_adjust=False, progress=False)
            if data.empty or 'Adj Close' not in data.columns:
                raise ValueError(f"No data or 'Adj Close' missing for {ticker}")
            price_data[ticker] = data["Adj Close"]
        except Exception as e:
            print(f"Error collecting cross-asset data for {ticker}: {e}")

    if not price_data:
        print("Warning: No cross-asset data collected.")
        return pd.DataFrame(index=pd.to_datetime([]))  # Create an empty DataFrame with DatetimeIndex

    cross_asset_df = pd.concat(price_data, axis=1)
    save_to_csv(cross_asset_df, "cross_asset_data")

    return cross_asset_df

# 7. ETF Metadata
def collect_etf_metadata():
    metadata = {}
    for ticker in ETF_UNIVERSE:
        print(f"Collecting metadata for {ticker}")
        try:
            ticker_obj = yf.Ticker(ticker)
            info = ticker_obj.info
            metadata[ticker] = {
                "Avg_Daily_Volume": info.get("averageDailyVolume10Day", np.nan),
                "Expense_Ratio": info.get("annualHoldingsTurnover", 0.15),
                "Bid_Ask_Spread": (info.get("bid", np.nan) - info.get("ask", np.nan)) if info.get("bid") and info.get("ask") else np.nan
            }
        except Exception as e:
            print(f"Error collecting metadata for {ticker}: {e}")

    if not metadata:
        print("Warning: No ETF metadata collected.")
        return pd.DataFrame()

    metadata_df = pd.DataFrame(metadata).T
    save_to_csv(metadata_df, "etf_metadata")

    return metadata_df

# Main function to collect all data
def collect_all_data():
    print("Starting data collection...")

    # Collect price/volume data once and cache
    price_df, volume_df, avg_daily_volume, failed_tickers, missing_data_tickers = collect_price_volume_data(ETF_UNIVERSE + CROSS_ASSETS)
    successful_tickers = [ticker for ticker in (ETF_UNIVERSE + CROSS_ASSETS) if ticker not in failed_tickers]

    factor_df = collect_factor_specific_data(price_df, [ticker for ticker in ETF_UNIVERSE if ticker not in failed_tickers])
    macro_df = collect_macro_data()
    volatility_df = collect_volatility_data()
    google_trends_df, credit_spread_df, sentiment_df = collect_alternative_data(price_df)
    cross_asset_df = collect_cross_asset_data()
    metadata_df = collect_etf_metadata()

    print("Data collection completed.")

    return {
        "price_data": price_df,
        "volume_data": volume_df,
        "avg_daily_volume": avg_daily_volume,
        "factor_data": factor_df,
        "macro_data": macro_df,
        "volatility_data": volatility_df,
        "google_trends_data": google_trends_df,
        "credit_spread_data": credit_spread_df,
        "sentiment_data": sentiment_df,
        "cross_asset_data": cross_asset_df,
        "metadata_data": metadata_df,
        "failed_tickers": failed_tickers,
        "missing_data_tickers": missing_data_tickers
    }

# Run the data collection
if __name__ == "__main__":
    data_dict = collect_all_data()

    print("\nSample Price Data:")
    print(data_dict["price_data"].tail())
    print("\nSample Macro Data:")
    print(data_dict["macro_data"].head())
    print("\nFailed Tickers (if any):")
    print(data_dict["failed_tickers"])
    print("\nMissing Data Tickers (if any):")
    print(data_dict["missing_data_tickers"])


Starting data collection...
Saved price_data to etf_data/price_data.csv
Saved volume_data to etf_data/volume_data.csv
Saved avg_daily_volume to etf_data/avg_daily_volume.csv
Calculating factor-specific data for MTUM
Calculating factor-specific data for VLUE
Calculating factor-specific data for QUAL
Calculating factor-specific data for USMV
Calculating factor-specific data for XLK
Calculating factor-specific data for XLV
Calculating factor-specific data for XLE
Calculating factor-specific data for XLU
Calculating factor-specific data for EEM
Calculating factor-specific data for EFA
Calculating factor-specific data for TLT
Calculating factor-specific data for IEF
Calculating factor-specific data for TIP
Calculating factor-specific data for BIL
Saved factor_data to etf_data/factor_data.csv
Saved macro_data to etf_data/macro_data.csv
Saved volatility_data to etf_data/volatility_data.csv
Saved credit_spreads to etf_data/credit_spreads.csv
Saved sentiment_data to etf_data/sentiment_data.csv


Feature Engineering -- Not done

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.decomposition import PCA
import os

# Check if running in Google Colab
try:
    from google.colab import files
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

def preprocess_and_engineer_features(data_dict):
    """
    Preprocess data and engineer features with explicit NaN handling and optimization
    """
    price_df = data_dict['price_data']
    macro_df = data_dict['macro_data']
    volatility_df = data_dict['volatility_data']
    volume_df = data_dict['volume_data']

    print("Price_df index:", price_df.index[:5])
    print("Macro_df index:", macro_df.index[:5])
    print("Volatility_df index:", volatility_df.index[:5])
    print("Price_df columns:", price_df.columns[:5])
    print("Macro_df columns:", macro_df.columns)

    features_df = pd.DataFrame(index=price_df.index)

    # 1. Momentum Features
    def calculate_momentum_features(ticker):
        data = price_df[ticker]['Adj Close'].squeeze().reindex(features_df.index, method='ffill')
        print(f"{ticker} Adj Close shape:", data.shape)

        features_df[f'{ticker}_12m_mom'] = data.pct_change(252)
        features_df[f'{ticker}_6m_mom'] = data.pct_change(126)
        features_df[f'{ticker}_3m_mom'] = data.pct_change(63)
        features_df[f'{ticker}_Mom_Accel'] = features_df[f'{ticker}_12m_mom'].diff(21) / 21

        sma200 = data.rolling(200).mean().reindex(features_df.index, method='ffill')
        features_df[f'{ticker}_SMA200'] = sma200

        data_aligned, sma200_aligned = data.align(sma200, join='inner', axis=0)
        features_df[f'{ticker}_Above_SMA'] = (data_aligned > sma200_aligned).astype(int).reindex(features_df.index, method='ffill')

        mom_12, mom_3 = features_df[f'{ticker}_12m_mom'].align(features_df[f'{ticker}_3m_mom'], join='inner')
        features_df[f'{ticker}_Mom_Div'] = (mom_12 - mom_3).reindex(features_df.index, method='ffill')

        return features_df

    # 2. Risk Features
    def calculate_risk_features(ticker):
        data = price_df[ticker]
        print(f"{ticker} data columns:", data.columns)
        returns = data['Adj Close'].squeeze().pct_change().reindex(features_df.index, method='ffill')
        features_df[f'{ticker}_Returns'] = returns
        features_df[f'{ticker}_Vol30'] = returns.rolling(30).std() * np.sqrt(252)

        spy_returns = price_df['SPY']['Adj Close'].squeeze().pct_change().reindex(features_df.index, method='ffill')
        returns, spy_returns = returns.align(spy_returns, join='inner', axis=0)
        print(f"{ticker} returns shape after align:", returns.shape)

        rolling_returns = pd.concat([returns, spy_returns], axis=1).dropna()
        if len(rolling_returns) >= 252:
            model = sm.OLS(rolling_returns.iloc[:, 0],
                          sm.add_constant(rolling_returns.iloc[:, 1])).fit()
            features_df[f'{ticker}_Beta252'] = model.params.iloc[1]

        covariance = returns.rolling(63).cov(spy_returns)
        spy_variance = spy_returns.rolling(63).var()
        print(f"{ticker} covariance shape:", covariance.shape)
        print(f"{ticker} spy_variance shape:", spy_variance.shape)
        beta63 = covariance.div(spy_variance).reindex(features_df.index, method='ffill')
        print(f"{ticker} Beta63 shape:", beta63.shape)
        features_df[f'{ticker}_Beta63'] = beta63

        features_df[f'{ticker}_CVaR'] = returns.rolling(252).apply(
            lambda x: np.percentile(x[x < np.percentile(x, 5)], 5) if len(x.dropna()) > 0 else np.nan
        )

        adj_close = data['Adj Close'].squeeze().reindex(features_df.index, method='ffill')
        features_df[f'{ticker}_Peak'] = adj_close.cummax()
        features_df[f'{ticker}_DD'] = (adj_close - features_df[f'{ticker}_Peak']) / features_df[f'{ticker}_Peak']
        features_df[f'{ticker}_Time_Underwater'] = (
            (adj_close < features_df[f'{ticker}_Peak']).astype(int).cumsum()
        )

        return features_df

    # 3. Macro Regime Features
    def calculate_macro_features():
        macro_aligned = macro_df.reindex(features_df.index, method='ffill')

        macro_aligned['YC_Slope'] = macro_aligned['10Y_2Y_Spread'].rolling(21).mean()
        macro_aligned['YC_Signal'] = (macro_aligned['YC_Slope'] > 0).astype(int)

        try:
            t10y3m = fred.get_series('T10Y3M', START_DATE, END_DATE)
            macro_aligned['T10Y3M'] = t10y3m.reindex(macro_aligned.index, method='ffill')
            macro_aligned['T10Y3M_Signal'] = (macro_aligned['T10Y3M'] > 0).astype(int)
        except:
            print("T10Y3M data not available")

        vix_aligned = volatility_df['VIX'].reindex(features_df.index, method='ffill')
        macro_aligned['VIX_Regime'] = pd.cut(
            vix_aligned,
            bins=[0, 12, 20, 30, 100],
            labels=[0, 1, 2, 3],
            include_lowest=True
        )

        macro_aligned['CPI_Mom'] = macro_aligned['CPI_YoY'].diff(21) / 21
        macro_aligned['PCE_Signal'] = (macro_aligned['CPI_YoY'] > 0.02).astype(int)

        return macro_aligned

    # 4. Relative Strength & Correlation Features
    def calculate_relative_features(ticker):
        spy_12m_mom = features_df['SPY_12m_mom']
        ticker_12m_mom = features_df[f'{ticker}_12m_mom']
        ticker_12m_mom, spy_12m_mom = ticker_12m_mom.align(spy_12m_mom, join='inner', axis=0)
        features_df[f'{ticker}_RS_SPY'] = (ticker_12m_mom - spy_12m_mom).reindex(features_df.index, method='ffill')

        spy_returns = price_df['SPY']['Adj Close'].squeeze().pct_change().reindex(features_df.index, method='ffill')
        ticker_returns = price_df[ticker]['Adj Close'].squeeze().pct_change().reindex(features_df.index, method='ffill')
        ticker_returns, spy_returns = ticker_returns.align(spy_returns, join='inner', axis=0)

        features_df[f'{ticker}_Corr_SPY'] = ticker_returns.rolling(63).corr(spy_returns)

        if ticker == 'MTUM':
            vlue_returns = price_df['VLUE']['Adj Close'].squeeze().pct_change().reindex(features_df.index, method='ffill')
            ticker_returns, vlue_returns = ticker_returns.align(vlue_returns, join='inner', axis=0)
            features_df['Corr_MTUM_VLUE'] = ticker_returns.rolling(63).corr(vlue_returns)

        return features_df

    # Process all tickers
    tickers = [t for t in ETF_UNIVERSE + CROSS_ASSETS if t in price_df.columns.get_level_values(0)]

    for ticker in tickers:
        print(f"Processing {ticker} - Momentum and Risk")
        features_df = calculate_momentum_features(ticker)
        features_df = calculate_risk_features(ticker)

    for ticker in tickers:
        print(f"Processing {ticker} - Relative Features")
        features_df = calculate_relative_features(ticker)

    macro_df = calculate_macro_features()
    features_df = features_df.join(macro_df, how='left')

    # Normalization and Dimensionality Reduction
    ticker_specific_cols = [
        '12m_mom', '6m_mom', '3m_mom', 'Mom_Accel', 'Vol30',
        'Beta63', 'CVaR', 'DD', 'RS_SPY', 'Corr_SPY'
    ]
    macro_cols = ['YC_Slope', 'VIX_Regime', 'CPI_Mom']

    feature_dict = {}
    for ticker in tickers:
        for col in ticker_specific_cols:
            feature_dict[f'{ticker}_{col}'] = features_df[f'{ticker}_{col}']
    for col in macro_cols:
        feature_dict[col] = features_df[col]

    feature_matrix = pd.DataFrame(feature_dict, index=features_df.index)

    print("Feature matrix NaN count before handling:", feature_matrix.isna().sum().sum())
    feature_matrix = feature_matrix.ffill().bfill()
    print("Feature matrix NaN count after ffill/bfill:", feature_matrix.isna().sum().sum())
    print("Columns with NaNs after ffill/bfill:", feature_matrix.columns[feature_matrix.isna().any()].tolist())
    feature_matrix = feature_matrix.fillna(0)
    print("Feature matrix shape after imputation:", feature_matrix.shape)

    if feature_matrix.empty:
        raise ValueError("Feature matrix is empty after processing. Check data availability.")

    scaler = StandardScaler()
    normalized_features = scaler.fit_transform(feature_matrix)
    normalized_df = pd.DataFrame(normalized_features, index=feature_matrix.index, columns=feature_matrix.columns)

    robust_scaler = RobustScaler()
    robust_features = robust_scaler.fit_transform(feature_matrix)
    robust_df = pd.DataFrame(robust_features, index=feature_matrix.index, columns=feature_matrix.columns)

    pca = PCA(n_components=15)
    pca_features = pca.fit_transform(normalized_features)
    pca_df = pd.DataFrame(pca_features, index=feature_matrix.index, columns=[f'PC_{i+1}' for i in range(15)])

    # Save datasets to CSV
    output_dir = "etf_features"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    raw_file = os.path.join(output_dir, "raw_features.csv")
    normalized_file = os.path.join(output_dir, "normalized_features.csv")
    robust_file = os.path.join(output_dir, "robust_features.csv")
    pca_file = os.path.join(output_dir, "pca_features.csv")

    features_df.to_csv(raw_file)
    normalized_df.to_csv(normalized_file)
    robust_df.to_csv(robust_file)
    pca_df.to_csv(pca_file)

    print(f"Saved raw features to: {raw_file}")
    print(f"Saved normalized features to: {normalized_file}")
    print(f"Saved robust features to: {robust_file}")
    print(f"Saved PCA features to: {pca_file}")

    # Combine all into a single dataset
    combined_df = pd.concat([features_df, normalized_df.add_suffix('_norm'),
                             robust_df.add_suffix('_robust'), pca_df], axis=1)
    combined_file = os.path.join(output_dir, "combined_features.csv")
    combined_df.to_csv(combined_file)
    print(f"Saved combined features to: {combined_file}")

    # Download if in Google Colab
    if IN_COLAB:
        #files.download(combined_file)
        print(f"Downloading {combined_file} in Google Colab")
    else:
        print(f"File saved locally at {combined_file}. Use your local file explorer to access it.")

    return {
        'raw_features': features_df,
        'normalized_features': normalized_df,
        'robust_features': robust_df,
        'pca_features': pca_df,
        'pca_explained_variance': pca.explained_variance_ratio_
    }

if __name__ == "__main__":
    feature_dict = preprocess_and_engineer_features(data_dict)

    print("\nSample Raw Features:")
    print(feature_dict['raw_features'].tail())
    print("\nSample Normalized Features:")
    print(feature_dict['normalized_features'].tail())
    print("\nExplained Variance Ratio from PCA:")
    print(feature_dict['pca_explained_variance'])

Price_df index: DatetimeIndex(['2010-01-04', '2010-01-05', '2010-01-06', '2010-01-07',
               '2010-01-08'],
              dtype='datetime64[ns]', name='Date', freq=None)
Macro_df index: DatetimeIndex(['2010-01-01', '2010-01-04', '2010-01-05', '2010-01-06',
               '2010-01-07'],
              dtype='datetime64[ns]', freq=None)
Volatility_df index: DatetimeIndex(['2010-01-04', '2010-01-05', '2010-01-06', '2010-01-07',
               '2010-01-08'],
              dtype='datetime64[ns]', name='Date', freq=None)
Price_df columns: MultiIndex([('MTUM',      'Open', 'MTUM'),
            ('MTUM',      'High', 'MTUM'),
            ('MTUM',       'Low', 'MTUM'),
            ('MTUM',     'Close', 'MTUM'),
            ('MTUM', 'Adj Close', 'MTUM')],
           names=[None, 'Price', 'Ticker'])
Macro_df columns: Index(['CPI_YoY', '10Y_2Y_Spread', 'Unemployment_Rate', 'TED_Spread',
       'Industrial_Production'],
      dtype='object')
Processing MTUM - Momentum and Risk
MTUM Adj Close

<ipython-input-35-45b8aea9fafb>:77: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  features_df[f'{ticker}_Beta63'] = beta63
<ipython-input-35-45b8aea9fafb>:79: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  features_df[f'{ticker}_CVaR'] = returns.rolling(252).apply(
<ipython-input-35-45b8aea9fafb>:84: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-frag

Processing XLU - Momentum and Risk
XLU Adj Close shape: (3816,)
XLU data columns: MultiIndex([(     'Open', 'XLU'),
            (     'High', 'XLU'),
            (      'Low', 'XLU'),
            (    'Close', 'XLU'),
            ('Adj Close', 'XLU')],
           names=['Price', 'Ticker'])
XLU returns shape after align: (3816,)
XLU covariance shape: (3816,)
XLU spy_variance shape: (3816,)
XLU Beta63 shape: (3816,)


<ipython-input-35-45b8aea9fafb>:79: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  features_df[f'{ticker}_CVaR'] = returns.rolling(252).apply(
<ipython-input-35-45b8aea9fafb>:84: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  features_df[f'{ticker}_Peak'] = adj_close.cummax()
<ipython-input-35-45b8aea9fafb>:85: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get

Processing EEM - Momentum and Risk
EEM Adj Close shape: (3816,)
EEM data columns: MultiIndex([(     'Open', 'EEM'),
            (     'High', 'EEM'),
            (      'Low', 'EEM'),
            (    'Close', 'EEM'),
            ('Adj Close', 'EEM')],
           names=['Price', 'Ticker'])
EEM returns shape after align: (3816,)
EEM covariance shape: (3816,)
EEM spy_variance shape: (3816,)
EEM Beta63 shape: (3816,)


...

<ipython-input-35-45b8aea9fafb>:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  features_df[f'{ticker}_12m_mom'] = data.pct_change(252)
<ipython-input-35-45b8aea9fafb>:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  features_df[f'{ticker}_6m_mom'] = data.pct_change(126)
<ipython-input-35-45b8aea9fafb>:39: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get

Processing MTUM - Relative Features
Processing VLUE - Relative Features
Processing QUAL - Relative Features
Processing USMV - Relative Features
Processing XLK - Relative Features
Processing XLV - Relative Features
Processing XLE - Relative Features
Processing XLU - Relative Features
Processing EEM - Relative Features
Processing EFA - Relative Features
Processing TLT - Relative Features
Processing IEF - Relative Features
Processing TIP - Relative Features
Processing BIL - Relative Features
Processing SPY - Relative Features
Processing TLT - Relative Features
Processing GLD - Relative Features
Feature matrix NaN count before handling: 56073
Feature matrix NaN count after ffill/bfill: 3816
Columns with NaNs after ffill/bfill: ['VIX_Regime']
Feature matrix shape after imputation: (3816, 163)
Saved raw features to: etf_features/raw_features.csv
Saved normalized features to: etf_features/normalized_features.csv
Saved robust features to: etf_features/robust_features.csv
Saved PCA features to:

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Sample Raw Features:
            MTUM_12m_mom  MTUM_6m_mom  MTUM_3m_mom  MTUM_Mom_Accel  \
Date                                                                 
2025-02-27      0.196872     0.099977     0.001373       -0.004806   
2025-02-28      0.218286     0.116638     0.022050       -0.004086   
2025-03-03      0.198050     0.106013    -0.005388       -0.005355   
2025-03-04      0.160908     0.085318    -0.021281       -0.007044   
2025-03-05      0.148806     0.083357    -0.014601       -0.008747   

            MTUM_SMA200  MTUM_Above_SMA  MTUM_Mom_Div  MTUM_Returns  \
Date                                                                  
2025-02-27   202.255784               1      0.195499     -0.017140   
2025-02-28   202.430974               1      0.196236      0.018512   
2025-03-03   202.590432               1      0.203438     -0.018679   
2025-03-04   202.721930               1      0.182188     -0.020574   
2025-03-05   202.845421               1      0.163407      0.